## Tensorflow for machine learning
* writing images to tfrecords (binary) files for faster read in learning step

In [1]:
import glob
import os
import tensorflow as tf

In [3]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

In [4]:
sess.run(tf.train.match_filenames_once("/home/mbledowski/*.jpg"))

FailedPreconditionError: Attempting to use uninitialized value matching_filenames
	 [[Node: _send_matching_filenames_0 = _Send[T=DT_STRING, client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=-8030970676735188495, tensor_name="matching_filenames:0", _device="/job:localhost/replica:0/task:0/cpu:0"](matching_filenames)]]

In [ ]:
os.path.expanduser('~/Downloads/')

In [ ]:
image_filenames = glob.glob('./imagenet-dogs/Images/n02*/*.jpg')

In [ ]:
image_filenames[0:2]

In [ ]:
from itertools import groupby
from collections import defaultdict

In [ ]:
train_data = defaultdict(list)
test_data = defaultdict(list)

In [ ]:
# image_filename_with_breed = map(lambda x: (x.split("/")[3], x.split("/")[4]), image_filenames)
image_filename_with_breed = map(lambda x: (x.split("/")[3], x), image_filenames)

In [ ]:
image_filename_with_breed[0]

In [ ]:
for dog_breed, breed_images in groupby(image_filename_with_breed, lambda x: x[0]):
    # if dog_breed == 'n02085620-Chihuahua':
        for i, breed_image in enumerate(breed_images):
            if i % 5 == 0:
                test_data[dog_breed].append(breed_image)
            else:
                train_data[dog_breed].append(breed_image)

In [ ]:
# test_data.keys()
test_data['n02086910-papillon'][0:2]

In [ ]:
# train_data.keys()

In [ ]:
def write_records_file(dataset, record_location):

    sess = tf.Session()

    writer = None

    current_index = 0
    for breed, images_filenames in dataset.items():
        print breed, current_index
        for image_filename in images_filenames:
            if current_index % 100 == 0:
                if writer:
                    writer.close()

                record_filename = "{record_location}-{current_index}.tfrecords".format(
                    record_location=record_location,
                    current_index=current_index)

                writer = tf.python_io.TFRecordWriter(record_filename)

            current_index += 1

            image_file = tf.read_file(image_filename[1])

            try:
                image = tf.image.decode_jpeg(image_file)
            except:
                print(image_filename)
                continue

            # convert to gray scale and resize images
            grayscale_image = tf.image.rgb_to_grayscale(image)        
            resized_image = tf.image.resize_images(grayscale_image, size = [250, 151])

            image_bytes = sess.run(tf.cast(resized_image, tf.uint8)).tobytes()            
            image_label = breed.encode('utf-8')

            example = tf.train.Example(features=tf.train.Features(feature={
                        'label': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_label])),
                        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))
                    }))

            writer.write(example.SerializeToString())

        writer.close()

In [ ]:
# for breed, images_filenames in test_data.items():
#     if breed == 'n02086910-papillon':
#         for image_filename in images_filenames:
#             print image_filename

In [ ]:
# runf function for both data files
write_records_file(test_data, "./output/testing-images/testing-image")

In [ ]:
write_records_file(train_data, "./output/training-images/train-image")

In [ ]:
k = train_data.keys()[0]
train_data[k][0][1]

In [ ]:
# t1 = (1,2)
# t2 = (3,4)
# from itertools import product
# for a, b in product(t1,t2):
#     print a,b